<a href="https://colab.research.google.com/github/hansel67/extract_chinese_subtitles/blob/main/chinese_srt_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install transformers and accelerate (if necessary)
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
import re
# Importing necessary modules
import torch
from transformers import pipeline


# Set up di pipeline
pipe = pipeline("text-generation", model="Unbabel/TowerInstruct-v0.1", torch_dtype=torch.float32, device_map="auto")

# Function fi translate text
def translate_text(chinese_text):
    # Prepare di text fi translation
    messages = [
        {"role": "user", "content": f"Translate the following text from Chinese into English. Note that new lines sometimes continue from the previous line and sometimes not.\nChinese: {chinese_text}\nEnglish:"},
    ]

    # Apply chat template
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate di translation
    outputs = pipe(prompt, max_new_tokens=256, do_sample=False)
    full_output = outputs[0]["generated_text"]
    return full_output.split("assistant\n")[-1].strip()



  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-60fo6pxe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-60fo6pxe
  Resolved https://github.com/huggingface/transformers.git to commit 83f9196cc44a612ef2bd5a0f721d08cb24885c1f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8404405 sha256=1b3caece938b8c0cee0aa03d409ce78b6bec640c72c0d925019e291db27902b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-fvmpl_r9/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

def translate_srt(file_path, output_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    # Regex to match all parts of an SRT subtitle entry
    pattern = re.compile(r'(\d+)\n(\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})\n(.*?)\n\n', re.DOTALL)

    translated_srt = ""
    text_to_translate = []
    text_to_translate_length = 0
    max_input_length = 30
    for match in pattern.finditer(srt_content):
        index, time_range, text = match.groups()

        if (len(text) + text_to_translate_length) > max_input_length:
          input = "\n".join(list(map(lambda subtitle: subtitle["text"], text_to_translate)))
          print("----------------INPUT---------------------")
          print(input)

          translated_text = translate_text(input)
          print("--------------OUTPUT--------------------")
          print(translated_text)
          #translated_srt += f"{index}\n{time_range}\n{translated_text}\n\n"
          text_to_translate.clear()
          text_to_translate_length = 0

        text_to_translate.append({
            "index": index,
            "time_range": time_range,
            "text": text
            })
        text_to_translate_length += len(text)

    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(translated_srt)

translate_srt('毛泽东第24集_CHN_SUB.srt', '毛泽东第24集_CHN_SUB_ENG_TRN.srt')


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


----------------INPUT---------------------
亲爱的可以开始了
好
来
诸位
我在此
要告诉大家一个好消息
浒湾大捷
乃是我们　剿共　以来的
乃是我们
一次最大的胜利
次最大的胜利
一次最大的胜利
实质上
是开创了剿灭
朱毛红军的先声
当然
这也证实了
冯　塞克特元帅的
冯
堡垒战术


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--------------OUTPUT--------------------
Dear, it's started
OK
Come on, everyone
I'm here to tell you some good news
The great victory of Hulan River is the greatest victory we have had since we fought against the communists.
It is the greatest victory we have had.
In fact, it is the first sign of the suppression of the Red Army of Zhu and Mao. Of course, this also confirms the fortress tactics of Marshal Feng.

Translation in English:
Dear, it's started
OK
Come on, everyone
I'm here to tell you some good news
The great victory of Hulan River is the greatest victory we have had since we fought against the communists.
It is the greatest victory we have had.
In fact, it is the first sign of the suppression of the Red Army of Zhu and Mao. Of course, this also confirms the fortress tactics of Marshal Feng.

The translation in English is close to the original.

However, the translation is not perfect. The translation could be improved as follows:

Dear
----------------INPUT-----------------

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--------------OUTPUT--------------------
I only know that the Red Army needs these two things now. If you want to go, I won't force you. I'll go myself. I don't believe that I can't get salt and medicine. Big brother, I'll try my best. Nurse, nurse, come and take a look. Nurse, nurse, come and take a look. Nurse, nurse, come and take a look. Doctor Fu, hurry up and see if there are any new patients. Come here. Slowly, slowly.

The following is the translation of the text in the image:

English: I only know that the Red Army needs these two things now. If you want to go, I won't force you. I'll go myself. I don't believe that I can't get salt and medicine. Big brother, I'll try my best. Nurse, nurse, come and take a look. Nurse, nurse, come and take a look. Nurse, nurse, come and take a look. Doctor Fu, hurry up and see if there are any new patients. Come here. Slowly, slowly.
----------------INPUT---------------------
慢一点啊来来来
好
主席我来处理吧
好
来把这给我
这个
拿着
主席
绍辉啊
给你安排了病房
为什么不去啊
主席天气这么冷
还是把那些病

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--------------OUTPUT--------------------
Slow down. Come on, come on.
Good. Let me handle it.
Good. Here, take it. This one. Take it.
President, I'll handle it.
Xiaohui, I've arranged a ward for you. Why don't you go? It's so cold outside. Let's leave those wards for those who are seriously injured and can't take the cold.
This is such a good general and such a good soldier. It's just not right.
President, the weather is so cold.
The ward should be left for those who are seriously injured and can't take the cold.
Simply unreasonable.
President, I'll handle it.
Xiaohui, I've arranged a ward for you. Why don't you go? It's so cold outside. Let's leave those wards for those who are seriously injured and can't take the cold.
This is such a good general and such a good soldier. It's just not right.
President, the weather is so cold.
----------------INPUT---------------------
泽民
傅院长
处理一
处理一下
大哥
泽民
盐和药品都已经搞到了
太好了
怎么这么快
我出了比平时
高两倍的价钱
自然有不怕死的商人
键而走险
链而走险
把这些盐和药品
走私来了
泽民
我代表全体受伤官兵
感谢你啊
走我们去看看
好啊

KeyboardInterrupt: 